# Binary compound formation energy prediction example

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/a-ws-m/unlockNN/blob/master/examples/formation_energies.ipynb)

This notebook demonstrates how to create a probabilistic model for predicting
formation energies of binary compounds with a quantified uncertainty.


In [ ]:
!pip install unlocknn

In [1]:
import shutil
from pathlib import Path
from pprint import pprint

import pandas as pd
from megnet.models import MEGNetModel
from tensorflow.keras.callbacks import TensorBoard
from unlocknn.download import load_data
from unlocknn.model import MEGNetProbModel
from unlocknn.metrics import evaluate_uq_metrics


In [2]:
THIS_DIR = Path(".").parent
CONFIG_FILE = THIS_DIR / ".config"

MODEL_SAVE_DIR: Path = THIS_DIR / "binary_e_form_model"
LOG_DIR = THIS_DIR / "logs"
BATCH_SIZE: int = 128
NUM_INDUCING_POINTS: int = 500
OVERWRITE: bool = True
TRAINING_RATIO: float = 0.8

if OVERWRITE:
    for directory in [MODEL_SAVE_DIR, LOG_DIR]:
        if directory.exists():
            shutil.rmtree(directory)


# Data gathering

Here we download binary compounds that lie on the convex hull from the Materials
Project, then split them into training and validation subsets.


In [3]:
full_df = load_data("binary_e_form")
full_df.head()

,structure,formation_energy_per_atom
0,"[[ 1.982598 -4.08421341 3.2051745 ] La, [1....",-0.737439
1,"[[0. 0. 0.] Fe, [1.880473 1.880473 1.880473] H]",-0.068482
2,"[[1.572998 0. 0. ] Ta, [0. ...",-0.773151
3,"[[0. 0. 7.42288687] Hf, [0. ...",-0.177707
4,"[[ 1.823716 -3.94193291 3.47897025] Tm, [1....",-0.905038


In [4]:
num_training = int(TRAINING_RATIO * len(full_df.index))
train_df = full_df[:num_training]
val_df = full_df[num_training:]

print(f"{num_training} training samples, {len(val_df.index)} validation samples.")

train_structs = train_df["structure"]
val_structs = val_df["structure"]

train_targets = train_df["formation_energy_per_atom"]
val_targets = val_df["formation_energy_per_atom"]


4217 training samples, 1055 validation samples.


# Model creation

Now we load the `MEGNet` 2019 formation energies model, then convert this to a
probabilistic model.


In [5]:
meg_model = MEGNetModel.from_mvl_models("Eform_MP_2019")


INFO:megnet.utils.models:Package-level mvl_models not included, trying temperary mvl_models downloads..
INFO:megnet.utils.models:Model found in local mvl_models path


Instructions for updating:
The `validate_indices` argument has no effect. Indices are always validated on CPU and never validated on GPU.


Instructions for updating:
The `validate_indices` argument has no effect. Indices are always validated on CPU and never validated on GPU.


In [6]:
kl_weight = BATCH_SIZE / num_training

prob_model = MEGNetProbModel(
    meg_model=meg_model,
    num_inducing_points=NUM_INDUCING_POINTS,
    kl_weight=kl_weight,
)


Instructions for updating:
`jitter` is deprecated; please use `marginal_fn` directly.


Instructions for updating:
`jitter` is deprecated; please use `marginal_fn` directly.
/home/awsm/anaconda3/envs/unlockNN/lib/python3.8/site-packages/tensorflow_probability/python/distributions/gaussian_process.py:363: UserWarning: Unable to detect statically whether the number of index_points is 1. As a result, defaulting to treating the marginal GP at `index_points` as a multivariate Gaussian. This makes some methods, like `cdf` unavailable.
  warnings.warn(


# Train the uncertainty quantifier

Now we train the model. By default, the `MEGNet` (NN) layers of the model are
frozen after initialization. Therefore, when we call `prob_model.train()`, the
only layers that are optimized are the `VariationalGaussianProcess` (VGP) and the
`BatchNormalization` layer (`Norm`) that feeds into it.

After this initial training, we unfreeze _all_ the layers and train the full model simulateously.


In [7]:
tb_callback_1 = TensorBoard(log_dir=LOG_DIR / "vgp_training", write_graph=False)
tb_callback_2 = TensorBoard(log_dir=LOG_DIR / "fine_tuning", write_graph=False)


In [8]:
%load_ext tensorboard
%tensorboard --logdir logs

In [9]:
prob_model.train(
    train_structs,
    train_targets,
    epochs=50,
    val_inputs=val_structs,
    val_targets=val_targets,
    callbacks=[tb_callback_1],
)
prob_model.save(MODEL_SAVE_DIR)


Epoch 1/50


/home/awsm/anaconda3/envs/unlockNN/lib/python3.8/site-packages/tensorflow_probability/python/distributions/gaussian_process.py:363: UserWarning: Unable to detect statically whether the number of index_points is 1. As a result, defaulting to treating the marginal GP at `index_points` as a multivariate Gaussian. This makes some methods, like `cdf` unavailable.
  warnings.warn(
/home/awsm/anaconda3/envs/unlockNN/lib/python3.8/site-packages/tensorflow/python/framework/indexed_slices.py:447: UserWarning: Converting sparse IndexedSlices(IndexedSlices(indices=Tensor("gradient_tape/model/set2_set_1/Reshape_9:0", shape=(None,), dtype=int32), values=Tensor("gradient_tape/model/set2_set_1/Reshape_8:0", shape=(None,), dtype=float32), dense_shape=Tensor("gradient_tape/model/set2_set_1/Cast:0", shape=(1,), dtype=int32))) to a dense Tensor of unknown shape. This may consume a large amount of memory.
  warnings.warn(
/home/awsm/anaconda3/envs/unlockNN/lib/python3.8/site-packages/tensorflow/python/fram

33/33 - 19s - loss: 2044082.0000 - val_loss: 1888069.7500
Epoch 2/50
33/33 - 9s - loss: 1894192.7500 - val_loss: 1713764.2500
Epoch 3/50
33/33 - 9s - loss: 1620727.7500 - val_loss: 1368848.1250
Epoch 4/50
33/33 - 9s - loss: 1237875.7500 - val_loss: 978411.0000
Epoch 5/50
33/33 - 9s - loss: 891600.6250 - val_loss: 638471.2500
Epoch 6/50
33/33 - 9s - loss: 592225.8125 - val_loss: 361983.4688
Epoch 7/50
33/33 - 9s - loss: 353038.8125 - val_loss: 183462.9531
Epoch 8/50
33/33 - 9s - loss: 222249.0156 - val_loss: 110128.0234
Epoch 9/50
33/33 - 9s - loss: 161658.2031 - val_loss: 82292.2188
Epoch 10/50
33/33 - 9s - loss: 121945.1328 - val_loss: 68647.5547
Epoch 11/50
33/33 - 9s - loss: 99813.3750 - val_loss: 64859.9648
Epoch 12/50
33/33 - 9s - loss: 85247.1797 - val_loss: 50290.5156
Epoch 13/50
33/33 - 9s - loss: 85243.5938 - val_loss: 48014.1875
Epoch 14/50
33/33 - 9s - loss: 68791.3438 - val_loss: 41188.8320
Epoch 15/50
33/33 - 9s - loss: 53726.7227 - val_loss: 37138.2227
Epoch 16/50
33/33 -

/home/awsm/anaconda3/envs/unlockNN/lib/python3.8/site-packages/tensorflow/python/keras/utils/generic_utils.py:494: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  warnings.warn('Custom mask layers require a config and must override '
INFO:tensorflow:Assets written to: binary_e_form_model/megnet/assets


INFO:tensorflow:Assets written to: binary_e_form_model/nn/assets


/home/awsm/anaconda3/envs/unlockNN/lib/python3.8/site-packages/tensorflow/python/keras/utils/generic_utils.py:494: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  warnings.warn('Custom mask layers require a config and must override '
INFO:tensorflow:Assets written to: binary_e_form_model/nn/assets


In [10]:
prob_model.set_frozen(["NN", "VGP"], freeze=False)


In [11]:
prob_model.train(
    train_structs,
    train_targets,
    epochs=50,
    val_inputs=val_structs,
    val_targets=val_targets,
    callbacks=[tb_callback_2],
)
prob_model.save(MODEL_SAVE_DIR)


Epoch 1/50


/home/awsm/anaconda3/envs/unlockNN/lib/python3.8/site-packages/tensorflow_probability/python/distributions/gaussian_process.py:363: UserWarning: Unable to detect statically whether the number of index_points is 1. As a result, defaulting to treating the marginal GP at `index_points` as a multivariate Gaussian. This makes some methods, like `cdf` unavailable.
  warnings.warn(
/home/awsm/anaconda3/envs/unlockNN/lib/python3.8/site-packages/tensorflow/python/framework/indexed_slices.py:447: UserWarning: Converting sparse IndexedSlices(IndexedSlices(indices=Tensor("gradient_tape/model/set2_set_1/Reshape_9:0", shape=(None,), dtype=int32), values=Tensor("gradient_tape/model/set2_set_1/Reshape_8:0", shape=(None,), dtype=float32), dense_shape=Tensor("gradient_tape/model/set2_set_1/Cast:0", shape=(1,), dtype=int32))) to a dense Tensor of unknown shape. This may consume a large amount of memory.
  warnings.warn(
/home/awsm/anaconda3/envs/unlockNN/lib/python3.8/site-packages/tensorflow/python/fram

33/33 - 22s - loss: 56607.2461 - val_loss: 239531.3906
Epoch 2/50
33/33 - 9s - loss: 32880.5742 - val_loss: 41553.4414
Epoch 3/50
33/33 - 9s - loss: 28333.7891 - val_loss: 37830.7969
Epoch 4/50
33/33 - 9s - loss: 21829.8965 - val_loss: 22037.8613
Epoch 5/50
33/33 - 9s - loss: 23603.4180 - val_loss: 16792.0098
Epoch 6/50
33/33 - 9s - loss: 19261.4473 - val_loss: 17440.0293
Epoch 7/50
33/33 - 9s - loss: 13590.6309 - val_loss: 15088.8799
Epoch 8/50
33/33 - 9s - loss: 14142.5850 - val_loss: 11170.5479
Epoch 9/50
33/33 - 9s - loss: 11774.4180 - val_loss: 30299.1328
Epoch 10/50
33/33 - 9s - loss: 12263.8896 - val_loss: 20215.4316
Epoch 11/50
33/33 - 9s - loss: 14102.6650 - val_loss: 14102.4971
Epoch 12/50
33/33 - 9s - loss: 10196.0801 - val_loss: 12292.2920
Epoch 13/50
33/33 - 9s - loss: 10220.3564 - val_loss: 10971.9023
Epoch 14/50
33/33 - 9s - loss: 8508.3301 - val_loss: 9019.6621
Epoch 15/50
33/33 - 9s - loss: 8402.4307 - val_loss: 8551.6201
Epoch 16/50
33/33 - 9s - loss: 8860.3662 - val_

/home/awsm/anaconda3/envs/unlockNN/lib/python3.8/site-packages/tensorflow/python/keras/utils/generic_utils.py:494: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  warnings.warn('Custom mask layers require a config and must override '
INFO:tensorflow:Assets written to: binary_e_form_model/megnet/assets


INFO:tensorflow:Assets written to: binary_e_form_model/nn/assets


/home/awsm/anaconda3/envs/unlockNN/lib/python3.8/site-packages/tensorflow/python/keras/utils/generic_utils.py:494: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  warnings.warn('Custom mask layers require a config and must override '
INFO:tensorflow:Assets written to: binary_e_form_model/nn/assets


# Model evaluation

Finally, we'll evaluate model metrics and make some sample predictions! Note that the predictions give predicted values and standard deviations. The standard deviations can then be converted to an uncertainty;
in this example, we'll take the uncertainty as twice the standard deviation, which will give us the 95% confidence interval (see <https://en.wikipedia.org/wiki/68%E2%80%9395%E2%80%9399.7_rule>).


In [12]:
example_structs = val_structs[:10].tolist()
example_targets = val_targets[:10].tolist()

predicted, stddevs = prob_model.predict(example_structs)
uncerts = 2 * stddevs


/home/awsm/anaconda3/envs/unlockNN/lib/python3.8/site-packages/tensorflow_probability/python/distributions/gaussian_process.py:363: UserWarning: Unable to detect statically whether the number of index_points is 1. As a result, defaulting to treating the marginal GP at `index_points` as a multivariate Gaussian. This makes some methods, like `cdf` unavailable.
  warnings.warn(


In [13]:
pd.DataFrame(
    {
        "Composition": [struct.composition.reduced_formula for struct in example_structs],
        "Formation energy per atom / eV": example_targets,
        "Predicted / eV": [
            f"{pred:.2f} ± {uncert:.2f}" for pred, uncert in zip(predicted, uncerts)
        ],
    }
)


,Composition,Formation energy per atom / eV,Predicted / eV
0,Zr2Cu,-0.132384,-0.12 ± 0.04
1,NbRh,-0.401313,-0.47 ± 0.03
2,Cu3Ge,-0.005707,-0.05 ± 0.05
3,Pr3In,-0.273232,-0.14 ± 0.07
4,InS,-0.742895,-0.74 ± 0.04
5,TmPb3,-0.215892,-0.19 ± 0.04
6,InNi,-0.174754,-0.22 ± 0.04
7,GdGe,-0.857117,-0.87 ± 0.08
8,GdTl,-0.380423,-0.38 ± 0.03
9,HoTl3,-0.215986,-0.19 ± 0.04


In [14]:
val_metrics = evaluate_uq_metrics(prob_model, val_structs, val_targets)
train_metrics = evaluate_uq_metrics(prob_model, train_structs, train_targets)

print("Validation metrics:")
pprint(val_metrics)
print("Training metrics:")
pprint(train_metrics)


/home/awsm/anaconda3/envs/unlockNN/lib/python3.8/site-packages/tensorflow_probability/python/distributions/gaussian_process.py:363: UserWarning: Unable to detect statically whether the number of index_points is 1. As a result, defaulting to treating the marginal GP at `index_points` as a multivariate Gaussian. This makes some methods, like `cdf` unavailable.
  warnings.warn(


Validation metrics:
{'mae': 0.04725769517605442,
 'mse': 0.005525025199049348,
 'nll': 846.6881418477113,
 'rmse': 0.07433051324354856,
 'sharpness': 0.03184407506259326,
 'variation': 0.560193233222921}
Training metrics:
{'mae': 0.0263804768548787,
 'mse': 0.0017778792658559221,
 'nll': -8528.868566316694,
 'rmse': 0.04216490561896139,
 'sharpness': 0.03232312160322517,
 'variation': 0.5973602101569429}
